In [ ]:
import sqlParser
import os
from openai import OpenAI
from dotenv import load_dotenv

In [3]:
with open("sample.sql", "r", encoding="utf8") as sql_file:
        table_info = sqlParser.parse_sql(sql_file.read().strip())
        sqlParser.generate_JSON(table_info)

In [6]:
Client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
    base_url=os.environ.get("OPENAI_API_BASE")
)

def get_completion(prompt, model="gpt-3.5-turbo"):
    message = [
        {
            "role": "system",
            "content": 'For a given ER design text or image, use json to describe entities, attributes, and relationships between entities (one_to-one, many_to_one, or many_to_many). For example: {"ERD": [{"name": "Person", "attributes": ["id", "name"], "relationships": []}, {"name": "Passport", "attributes": ["id", "number", "person_id"], "relationships": [{"related_entity": "Person", "type": "one_to_one"}]}]}'
        },
        {
            "role": "user",
            "content": prompt,
        }
    ]

    response = Client.chat.completions.create(
        messages=message,
        model=model,
    )

    return response.choices[0].message.content

In [ ]:
prompt = """
- 学生(Student)
    - 属性: 学号(ID)、姓名(Name)、年龄(Age)
- 课程(Course)
    - 属性: 课程编号(CourseID)、课程名称(CourseName)
- 选课(Enrollment)
    - 属性: 成绩(Grade)
    - 外键: 学号(StudentID)、课程编号(CourseID)
"""
out = get_completion(prompt)
print(out)

In [8]:
import json
data = json.loads(out)
with open('ERDFromChat.json', 'w+') as file:
        json.dump(data, file)

In [ ]:
# TODO: Compare ERDFromChat with ERDFromPython 
   
with open('ERDFromChat.json', 'r') as f:  
    chat_data = json.load(f)  

with open('ERDFromPython.json', 'r') as f:  
    python_data = json.load(f)